In [1]:
import os
import re
import logging
import sys
from datetime import datetime

LOG_DIR = r"D:\Work Related\Logger files"
os.makedirs(LOG_DIR, exist_ok=True)

class LoggerSetup:
    def __init__(self, customer_id):
        self.logger = self.setup_logger(customer_id)

    def setup_logger(self, customer_id):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_filename = os.path.join(LOG_DIR, f"customer_{customer_id}_{timestamp}.log")
        logger = logging.getLogger(f"Logger_{customer_id}")
        logger.setLevel(logging.DEBUG)

        if not logger.handlers:
            handler = logging.FileHandler(log_filename)
            handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
            logger.addHandler(handler)

        return logger

class Validator:
    def __init__(self, logger):
        self.logger = logger

    def validate(self, pattern, value, error_message):
        if not re.match(pattern, value):
            self.logger.error(error_message)
            raise Exception(sys.exc_info()[1])
        self.logger.info(f"Validation passed for value: {value}")
        return True

class CustomerValidation:
    used_customer_ids = set()

    def __init__(self):
        self.logger = None
        self.validator = None

    def validate_customer_id(self, customer_id):
        if not re.match("^[1-9][0-9]{4}$", customer_id):
            print("Invalid customer ID. Must be a 5-digit number starting with a non-zero digit.")
            return False

        if customer_id in self.used_customer_ids:
            print("This customer ID has already been used. Please enter a unique ID.")
            return False

        self.used_customer_ids.add(customer_id)
        logger_setup = LoggerSetup(customer_id)
        self.logger = logger_setup.logger
        self.validator = Validator(self.logger)
        self.logger.info(f"Starting validation for customer ID: {customer_id}")
        return True

    def collect_and_validate_details(self):
        try:
            self.validator.validate("^[A-z]*[a-z]$", input("Enter name: "), "Invalid name.")
            self.validator.validate("^[0-9]{2}-[0-9]{2}-[0-9]{4}$", input("Enter date of birth (DD-MM-YYYY): "), "Invalid DOB format.")
            self.validator.validate("^[6-9][0-9]{9}$", input("Enter mobile number: "), "Invalid mobile number.")
            self.validator.validate("^[a-z][a-zA-Z0-9._]*@[a-z]*.com$", input("Enter email: "), "Invalid email address.")
            self.validator.validate("^.+$", input("Enter address: "), "Address cannot be empty.")
            self.validator.validate("^(male|female|other)$", input("Enter gender (male/female/other): ").lower(), "Invalid gender.")
            self.validator.validate("^^[1-9][0-2]$", input("Enter number of people (Maximum 12): "), "Invalid number of people.")
        except Exception as e:
            print(f"Validation error: {e}")
            self.logger.error("Validation failed.")
            return False
        else:
            print("All details are valid!")
            self.logger.info("Validation successful.")
            return True

print("\nEnter customer details (or type 'exit' to quit):")
customer_validation = CustomerValidation()

while True:
    customer_id = input("Enter unique customer ID: ").strip()

    if customer_id.lower() == "exit":
        print("Exiting the program.")
        break

    if not customer_validation.validate_customer_id(customer_id):
        continue

    if customer_validation.collect_and_validate_details():
        print("Validation complete.")

    print("\n------------------------------------------------------")



Enter customer details (or type 'exit' to quit):
All details are valid!
Validation complete.

------------------------------------------------------
Validation error: None

------------------------------------------------------
This customer ID has already been used. Please enter a unique ID.
Exiting the program.
